# Tutorial on DER Hosting Capacity | <font color=red>Part 4</font>: Monte Carlo Assessment of PV Hosting Capacity of an Integrated MV-LV Network

## Introduction
This tutorial is aimed to help those who wish to do a Monte Carlo analysis of a MV-LV Network in OpenDSS using Python. Here, you will analyse an Australian MV-LV network. The network is called CRE21, and it is an Urban network with 79 distribution transformers. The network supplies electricity to 3300+ houses. 

The network configuration is given in figure 1. The triangle is the 66/22 kV transformer and each circle represents a distribution transformer. 

<img style="float: middle;" src="Network_CRE21.png">

**<center>Figure 1. The CRE21 Network Configuration</center>**

# Initialization
Here, the first steps related to importing the relevant libraries and setting the working environment are given below.

## Load libraries
Run the code below to load the libraries that will be used in this tutorial.

In [ ]:
import time
from matplotlib import pyplot as plt
import random
import numpy as np
import os

## Set the working path
Run the following code to set your working path. As an output you get the location in your computer.

In [ ]:
mydir =  os.getcwd()
print("The direction is located in the following path: mydir = %s\n" %mydir) 

## Set up  dss_engine
Before running the code, we need to set up the dss_engine.  
- Note: For better understanding of the following code, check the **dss_python tutorial** in the Team Tutorial folder.

In [ ]:
import dss
dss_engine = dss.DSS
DSSText = dss_engine.Text                                                      
DSSCircuit = dss_engine.ActiveCircuit                                            
DSSSolution = dss_engine.ActiveCircuit.Solution                                      
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue                                          
dss_engine.AllowForms = 0

# Network Modelling

## Basic definitions
Here the values which will be leter used are allocated to corresponding variables. 
- `Time_Resolution` in minutes. The 30 minute resolution is chosen for this tutorial.
- `Num_of_TimeStep`, which is the number of timesteps, based on the Time Resolution used
- `Num_of_DisTransformers` is the Number of Distribution Transformers and this value is also fixed
- `Num_of_HVlines` is the number of HV lines, 
- `Inverter_factor`, set and fixed to 1.0,
- `Voltage_max`, is the maximum value of the voltage in p.u.,

This is done for better understanding of the code, but note that all values used in this tutorial will be constant **i.e., you should NOT change them**.

In [ ]:
Time_Resolution = 30 #in minutes
Num_of_TimeStep = 48
Num_of_DisTransformers = 79
Num_of_HVlines = 649
Inverter_factor = 1.0
Voltage_max = 1.1

## Import Load and PV profiles
The data used for this tutorial are provided in the folder `/Datasets`. 
- `House data - 30 mins resolution.npy`: This numpy file contains the 342 demand profiles for residential customers for a whole year in 30-minute intervals
- `PV data - 30 mins resolution.npy`: This numpy file contains the PV generation profile for a whole year in 30-minute intervals 

In [ ]:
houseDataminutes = np.load(mydir + '//Datasets//House data - 30 mins resolution.npy') 
pvDataminutes = np.load(mydir + '//Datasets//PV data - 30 mins resolution.npy')  

Now that the data has been loaded, let's take a look at the dimensions of the numpy files.  
- Note: The format of the house data is `(Customers, Days, Readings)` and the PV profiles' data is `(Days, Readings)` which can be seen by running the code below.

In [ ]:
print("The shape of the house date is: ",houseDataminutes.shape) 
print("The shape of the PV profiles is: ", pvDataminutes.shape) 

## Define components
The `Master.txt` file, located in the `Network` folder, defines the frequency and base voltages, then it redirects to network components definitions: transformers, lines and loads etc,.

In [ ]:
DSSText.Command = 'Clear'   
DSSText.Command = 'Compile ' + mydir + '\\Network\\Master.txt'
DSSText.Command = 'Set VoltageBases=[66.0, 22.0, 0.400, 0.2309]'
DSSText.Command = 'calcv'  

## Install PV systems 
Here the PV systems are **installed** for every **residential** customer when the command `if phases==1` is executed.  
- However, at this step the PV systems are all initialized to zero (`kva=0` and `pmpp=0`) and are not enabled (as it can be seen from the code below, where `enabled=false`).

In [ ]:
Loadname = DSSCircuit.Loads.AllNames
for iPV_status in range(len(Loadname)):
    DSSCircuit.SetActiveElement('load.' + str(Loadname[iPV_status]))
    phases = int(DSSCircuit.ActiveCktElement.Properties('phases').Val)
    bus1 = DSSCircuit.ActiveCktElement.Properties('bus1').Val
    if phases == 1:
        DSSText.Command = 'new PVSystem.PV_' + str(Loadname[iPV_status])\
                        + ' phases=1'  \
                        + ' irradiance=1' \
                        + ' %cutin=0.05' \
                        + ' %cutout=0.05' \
                        + ' vmaxpu=1.5' \
                        + ' vminpu=0.5' \
                        + ' kva=0' \
                        + ' pmpp=0' \
                        + ' bus1=' + str(bus1) \
                        + ' pf=1' \
                        + ' enabled=false' \
                        + ' kv=0.23' \
                        + ' daily=pvshape1'\
                        + ' VarFollowInverter=True'  

# Function Definitions
**Important:** We highly recommend you first go through [Part.5 Simulation and Results](#004) and come back to this Functions Definitions session for a better understanding of the code structure. Just bear in mind that these sub functions has to be compiled before the main function starts.

<a id='001'></a>
## def  <font color=blue>Function_LoadProfileAllocation()</font>
- **Line [4-5]:** `cust` is named in the form of `hv_f0_lv0_f0_c0`, i.e. HVfeeder 0, LVtransformer 0, LVfeeder 0, Customer 0. What we need from the cust name is the LV transformer number, so we split the name to retrieve the information.
- **Line [6-11]:** Only for those 3-phase transformers, loads are randomly allocated to different phases(.1/.2/.3). And loads under 1-phase transformer are connected to phase.1 in default.
- **Line [13-17]:** This part checks if it is a single phase load, and if yes, assign new `bus1` and `daily` values. Noted that for commercial loads (3-phase), profiles are fixed, not randomly selected.

In [ ]:
def Function_LoadProfileAllocation():
    np.random.seed(iRandom)
    
    for icust, cust in enumerate(Loadname):    
        
        DSSCircuit.SetActiveElement('load.'+ cust) 
        phases = int(DSSCircuit.ActiveElement.Properties('phases').Val) # phase of load
        Cust_ran = np.random.randint(len(houseDataminutes))  
        
        if phases == 1: # residential load
            DSSCircuit.ActiveElement.Properties('daily').Val='Load_shape_' + str(Cust_ran)    
        else: # commercial load
            cust_split = cust.split('_')
            Transformername = cust_split[2]
            DSSCircuit.ActiveElement.Properties('daily').Val='Load_shape_Com_' + str(Transformername)

<a id='002'></a>
## def  <font color=blue>Function_PVAllocation()</font>  
Here, inverter control is disabled to improve simulation speed. Please uncomment the inverter part if you want to introduce this feature to your study.
- **Line [2]:** ` PVcustomer_number` corresponds to the penetration level.
- **Line [3-8]:** `Count` is used to count the PV system has been enabled. `PV_status_dct` records the 'enabled' status of each PV system.
- **Line [9-21]:** Each PV system has the same probability to be enabled by using the `if random.random()<...` condition. To make sure the penetration percent is correct, we need to check `Count` after each entire search, and break the loop when it conforms.


In [ ]:
def Function_PVAllocation():   
    PVcustomer_number=[0, 675, 1350, 2024, 2699, 3374]
    if penetration_list[iPenetration] == 0 :
        global Count, PV_status_dct
        Count = 0
        PV_status_dct = []
        for iPV_status in range(len(PVname)):
            PV_status_dct.append('false')
    else:
        while Count < PVcustomer_number[iPenetration]:
            for iPV_status in range(len(PVname)):
                if PV_status_dct[iPV_status] == 'false':
                    if random.random() < (penetration_list[iPenetration]-penetration_list[iPenetration-1])/(100-penetration_list[iPenetration-1]):
                        PV_status_dct[iPV_status] = 'true'                            
                        DSSCircuit.SetActiveElement('PVSystem.' + str(PVname[iPV_status]))
                        DSSCircuit.ActiveElement.Properties('kva').Val=str(Inverter_factor * PV_allocation[iPV_status])
                        DSSCircuit.ActiveElement.Properties('pmpp').Val=str(PV_allocation[iPV_status])
                        DSSCircuit.ActiveElement.Properties('enabled').Val='true'
                        Count = Count + 1
                        if Count == PVcustomer_number[iPenetration]:
                            break 
                            
#        if iRandom == 0:
#            DSSText.Command = 'New XYCurve.vw_curve'+ str(iPenetration)+' npts=4 Yarray=(1.0, 1.0, 0.2, 0.2) XArray=(0.5, 1.1, 1.13, 2.0)' 
#            DSSText.Command = 'New InvControl.InvPVCtrl'  + str(iPenetration) + ' mode=voltwatt voltwatt_curve=vw_curve'+ str(iPenetration) +' DeltaP_factor=0.05'             
#            DSSText.Command = 'calcv'
#            DSSText.Command = 'set maxcontroliter=1000'
#            DSSText.Command = 'set maxiterations=1000'
#            DSSText.Command = 'calcvoltagebases'         
#            DSSText.Command = 'calcv'

<a id='003'></a>
## def  <font color=blue>Function_iTimeSimulation()</font>
This 24-hour, 30-min resolution `iTimeSimulation` is just same as the one you did in **Deterministic Analysis**, so please refer to that tutorial for detailed explanation on the code.  

There is one thing needs highlight:
- **Line [140-148]:** After the whole simulation, the results are stored in ` Penetration_results_dct`. There are 8 indices under this dictionary, and they will be used in the Plots. For your convenience, here is a search table.  

| Index | Parameter | 
| :------: | :------ | 
| 0 | 100*np.amax(np.array(total_HVT)[:,2])/HV_Trans_Capacity |
| 1 | np.amax(np.array(DisTrans_Uti_Max)) |
| 2 | np.amax(np.array(Line_Utilise_max)) |
| 3 | 100*np.amax(np.array(Non_compliance))/len(Loadname) |
| 4 | np.sum(PV_Curtail) |
| 5 | np.sum(PV_Potential) |
| 6 | np.sum(PV_Potential, axis=0) |
| 7 | np.sum(PV_P, axis=0) |

In [ ]:
def Function_iTimeSimulation():
    for iTime in range(Num_of_TimeStep):
        if iTime == 0:
            DisTrans_Uti_Max = [] 
            Line_Utilise_max = []
            Non_compliance = []
            
            NodeName_dct = {}
            LVT_dct = {}
            LNE_dct = {}
            LDE_dct = {}
            
            LDE_bus1 = []
            LDE_bus1_index = []
            LDE_bus_main = []

            all_loads_voltages = []
            
            P_HVT = []
            Q_HVT = []
            S_HVT = []
            total_HVT = []   
                           
            PV_P = []
            PV_Q = []
            PV_Curtail = []
            PV_Potential = []
            PV_Curtail_Percent = []       
    
            all_node_names = DSSCircuit.AllNodeNames
            for iNodes in range(len(all_node_names)):
                NodeName_dct[all_node_names[iNodes]] = iNodes        
            for iTransformer in range(Num_of_DisTransformers):
                LVT_dct['LVT_%s' %iTransformer] = []
            for iLine in range(Num_of_HVlines):
                LNE_dct['LNE_%s' %iLine] = []
            for icust in range(len(Loadname)):
                LDE_dct[Loadname[icust]] = []
                DSSCircuit.SetActiveElement('load.'+Loadname[icust])
                phases = int(DSSCircuit.ActiveCktElement.Properties('phases').Val)
                bus1 = DSSCircuit.ActiveCktElement.Properties('bus1').Val
                LDE_bus_main.append(bus1)
                if phases == 1:
                    LDE_bus1.append(bus1)
                    LDE_bus1_index.append(all_node_names.index(bus1))
                elif phases == 3:
                    LDE_bus1.append(bus1.strip('.1.2.3')+ '.1')
                    LDE_bus1_index.append(all_node_names.index(bus1.strip('.1.2.3')+ '.1'))
                    LDE_bus1.append(bus1.strip('.1.2.3')+ '.2')
                    LDE_bus1_index.append(all_node_names.index(bus1.strip('.1.2.3')+ '.2'))
                    LDE_bus1.append(bus1.strip('.1.2.3')+ '.3')
                    LDE_bus1_index.append(all_node_names.index(bus1.strip('.1.2.3')+ '.3'))  

        
        DSSSolution.Solve()
        
        
        DSSCircuit.SetActiveElement('Transformer.hv_head_tx0')
        HV_Trans_Capacity = float(DSSCircuit.ActiveCktElement.Properties('kVAs').Val.strip('[').strip(']').split(',')[0])
        P_HVT_temp = (DSSCircuit.ActiveCktElement.Powers[0] + DSSCircuit.ActiveCktElement.Powers[2] + DSSCircuit.ActiveCktElement.Powers[4])
        P_HVT.append(P_HVT_temp) 
        Q_HVT_temp = (DSSCircuit.ActiveCktElement.Powers[1] + DSSCircuit.ActiveCktElement.Powers[3] + DSSCircuit.ActiveCktElement.Powers[5])
        Q_HVT.append(Q_HVT_temp)
        S_HVT_temp = np.sqrt(P_HVT_temp**2 + Q_HVT_temp**2)
        S_HVT.append(S_HVT_temp)
        total_HVT.append([P_HVT_temp, Q_HVT_temp, S_HVT_temp])
        
        
        DisTrans_Uti_Max_temp =[]
        for iTransformer in range(Num_of_DisTransformers):
            DSSCircuit.SetActiveElement('transformer.hv_f0_lv%s'%iTransformer + '_tx')    
            number_phases = int(DSSCircuit.ActiveElement.Properties('phases').Val)
            Trans_Capacity = float(DSSCircuit.ActiveCktElement.Properties('kVAs').Val.strip('[').strip(']').split(',')[0])
            if number_phases == 3:
                P1_LVT = DSSCircuit.ActiveCktElement.Powers[0] + DSSCircuit.ActiveCktElement.Powers[2] + DSSCircuit.ActiveCktElement.Powers[4]
                Q1_LVT = DSSCircuit.ActiveCktElement.Powers[1] + DSSCircuit.ActiveCktElement.Powers[3] + DSSCircuit.ActiveCktElement.Powers[5]
                S1_LVT = np.sqrt(P1_LVT**2 + Q1_LVT**2)
                P2_LVT = DSSCircuit.ActiveCktElement.Powers[8] + DSSCircuit.ActiveCktElement.Powers[10] + DSSCircuit.ActiveCktElement.Powers[12]
                Q2_LVT = DSSCircuit.ActiveCktElement.Powers[9] + DSSCircuit.ActiveCktElement.Powers[11] + DSSCircuit.ActiveCktElement.Powers[13]
                S2_LVT = np.sqrt(P2_LVT**2 + Q2_LVT**2)   
            if number_phases == 1:
                P1_LVT = DSSCircuit.ActiveCktElement.Powers[0] + DSSCircuit.ActiveCktElement.Powers[2] + DSSCircuit.ActiveCktElement.Powers[4]
                Q1_LVT = DSSCircuit.ActiveCktElement.Powers[1] + DSSCircuit.ActiveCktElement.Powers[3] + DSSCircuit.ActiveCktElement.Powers[5]
                S1_LVT = np.sqrt(P1_LVT**2 + Q1_LVT**2)
                P2_LVT = DSSCircuit.ActiveCktElement.Powers[6] + DSSCircuit.ActiveCktElement.Powers[8] + DSSCircuit.ActiveCktElement.Powers[10]
                Q2_LVT = DSSCircuit.ActiveCktElement.Powers[7] + DSSCircuit.ActiveCktElement.Powers[9] + DSSCircuit.ActiveCktElement.Powers[11]
                S2_LVT = np.sqrt(P2_LVT**2 + Q2_LVT**2)
            LVT_dct['LVT_%s' %iTransformer].append([P1_LVT, Q1_LVT, S1_LVT, P2_LVT, Q2_LVT, S2_LVT]) 
            DisTrans_Uti_Max_temp.append(100*S1_LVT/Trans_Capacity)
        DisTrans_Uti_Max.append(np.amax(np.array(DisTrans_Uti_Max_temp)))   
        
            
        Line_Utilise_max_temp = []
        for iLine in range(Num_of_HVlines):
            DSSCircuit.SetActiveElement('line.HV_F0_L%s'%iLine)
            I11 = DSSCircuit.ActiveCktElement.CurrentsMagAng[0]
            I12 = DSSCircuit.ActiveCktElement.CurrentsMagAng[2]
            I13 = DSSCircuit.ActiveCktElement.CurrentsMagAng[4]
            NormAmps = DSSCircuit.Lines.NormAmps
            I_sum = I11 + I12 + I13
            I_max = np.maximum(np.array(I11), np.array(I12), np.array(I13))
            LNE_dct['LNE_%s' %iLine].append([I11, I12, I13, NormAmps, 100*I_sum/(NormAmps*3), 100*I_max/NormAmps])
            Line_Utilise_max_temp.append(100*I_max/NormAmps)
        Line_Utilise_max.append(np.amax(np.array(Line_Utilise_max_temp)))


        LDE_temp = np.array(DSSCircuit.AllBusVmagPu)[LDE_bus1_index]
        all_loads_voltages.append(LDE_temp)
        Non_compliance.append(sum(i > Voltage_max for i in LDE_temp))
            

        PV_P_temp = []
        PV_Q_temp = []
        PV_Curtail_temp = []
        PV_Potential_temp = []
        PV_Curtail_Percent_temp = []                    
        PVname = DSSCircuit.PVSystems.AllNames
        if penetration_list[iPenetration]>0:
            for iPV in range(len(PVname)):
                DSSCircuit.SetActiveElement('PVsystem.'+ PVname[iPV])
                iPV_status=DSSCircuit.ActiveCktElement.Properties('enabled').Val
                if iPV_status=='true':
                    PV_size = DSSCircuit.ActiveElement.Properties('Pmpp').Val
                    pvoutput = (np.abs(DSSCircuit.ActiveElement.Powers[0]))
                    pvpotential = float(PV_size) * PV_potential[iTime]
                    pvcurtail = pvpotential - pvoutput
                    if pvcurtail < 0.01:
                        pvcurtail = 0
                    PV_P_temp.append(np.abs(DSSCircuit.ActiveElement.Powers[0]))
                    PV_Q_temp.append(DSSCircuit.ActiveElement.Powers[1])
                    PV_Potential_temp.append(pvpotential)
                    PV_Curtail_temp.append(pvcurtail)
                    PV_Curtail_Percent_temp.append(100*(pvcurtail)/(pvpotential + 0.0001))     
            PV_P.append(PV_P_temp)
            PV_Q.append(PV_Q_temp)
            PV_Potential.append(PV_Potential_temp)
            PV_Curtail.append(PV_Curtail_temp)
            PV_Curtail_Percent.append(PV_Curtail_Percent_temp)
        
    Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]].append \
                            ([100*np.amax(np.array(total_HVT)[:,2])/HV_Trans_Capacity, \
                              np.amax(np.array(DisTrans_Uti_Max)), \
                              np.amax(np.array(Line_Utilise_max)), \
                              100*np.amax(np.array(Non_compliance))/len(Loadname), \
                              np.sum(PV_Curtail), \
                              np.sum(PV_Potential), \
                              np.sum(PV_Potential, axis=0), \
                              np.sum(PV_P, axis=0)])

<a id='004'></a>
# Simulation and Results

## Set number of iterations
Here, you can change the number of iterations for Monte Carlo simulation, and with larger number, more cases are covered then you could observe more general results.
- Notice that without inverter control, each iteration takes around **1-min**, while with inverter control enabled, it gonna take around **10-min**.

In [ ]:
Num_Run = 2

## Create results dictionary
The results dictionary is created for each penetration level, and with the tag `penetration`, you are able to look into  particular case for actual outputs. 
- Notice that you are **NOT** encouraged to change the penetration list at first unless you understand the whole code and you can try to edit it in the sourcecode `Chapter4_MonteCarlo.py` file, otherwise the boxplots will show wrong results.

In [ ]:
penetration_list = [0, 20, 40, 60, 80, 100]
Penetration_results_dct = {}
for iPenetration in range(len(penetration_list)):
    Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]]= [] 

## Monte Carlo simulation
This part is the main function of Monte Carlo simulation, you will observe a lot of 'random' things for probabilistic analysis.

- **Line [5-13]: First randomization on day profile.** Any combination of load profile and PV profile may happen including the worst condition - the day with lowest load and the day with highest PV. Note: It's a season analysis in Southern Summer, during Day 0-62 and Day 335-365.  

- **Line [26]: Second randomization on load profile allocation.** Traceback to [function definition 4.1](#001), first the load phase is randomly chosen (only for 3-phase transformer) then the profile is also randomly selected from the housedata pool.  

- **Line [32-33, 35]: Third randomization on PV system allocation.** For each iteration, size of all PV systems are randomly assigned, and only a select of them are activated according to the penetration level. The method to allocate PV systems is introduced in [function definition 4.2](#002).

Congratulations! After all these long and tough random assignments, the network is ready now. Just like Deterministic Analysis you did before, the same 30-min resolution power flow analysis is executed (See [function definition 4.3](#003)).  
  
And the last step in `iRandom` loop is to reset all the PV systems, make sure everytime you start from 0% penetration.

In [ ]:
start_time = time.time() 


for iRandom in range(Num_Run):
    np.random.seed(iRandom)
    if np.random.randint(low=0, high=3) == 0:
        Day_Num = np.random.randint(low=335, high=365)
    else:
        Day_Num = np.random.randint(low=0, high=62)
    if np.random.randint(low=0, high=3) == 0:
        Day_Num_PV = np.random.randint(low=335, high=365)
    else:
        Day_Num_PV = np.random.randint(low=0, high=62) 
        
    print('The run number is %s.' %iRandom)
    print('Day number: %s, ' %Day_Num + 'PV Day number: %s.' %Day_Num_PV )
    
    for ii in range(len(houseDataminutes)):
        DSSCircuit.LoadShapes.Name= 'Load_shape_%s'%(ii)
        DSSCircuit.LoadShapes.Pmult=houseDataminutes[ii,Day_Num,:].tolist()  
    DSSCircuit.LoadShapes.Name='pvshape1'
    DSSCircuit.LoadShapes.Pmult=pvDataminutes[Day_Num_PV,:].tolist()    
    PV_potential =  pvDataminutes[Day_Num_PV,:] 


    Function_LoadProfileAllocation()


    PV_allocation = [] 
    PVname = DSSCircuit.PVSystems.AllNames
    np.random.seed(iRandom)
    for iPV in range(len(PVname)):
        PV_allocation.append(np.random.choice([2.5,3.5,5.5,8], p=[0.08,0.24,0.52,0.16]))        
    random.seed(iRandom)
    for iPenetration in range(len(penetration_list)):     
        Function_PVAllocation()
        DSSText.Command = 'Set ControlMode=statatic' 
        DSSText.Command = 'Reset'   
        DSSText.Command = 'Set Mode=daily number=1 stepsize=%s' %Time_Resolution +'m' 
        Function_iTimeSimulation()
        

    for iPV in range(len(PVname)): 
        DSSCircuit.SetActiveElement('PVSystem.'+str(PVname[iPV]))
        DSSCircuit.ActiveElement.Properties('enabled').Val='false'

        
print("Monte Carlo simulation time (in seconds) = ", time.time()-start_time) 

# Plots
By retrieving the results from `Penetration_results_dct`, you can now draw box plots and assess the network performance from different aspects. Fig.5 and Fig.6 are for PV curtailment, so if you play with the tutorial in a non-inverter control mode, you can only observe no curtailment for granted.  
  
- Note: The results are not representative, becasue `Num_Run = 10` is not big enough. We set a small number here to save the simulation time. The document provided in the same folder [Simulation_results_of_different_Num_Run.docx](Simulation_results_of_different_Num_Run.docx) shows the  results comparison of **10, 30, 100 and 200** times simulation. The number of iteration is a key factor of Monte Carlo simulation.

## Fig.1: 66/22 kV transformer utilization level

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.rc('font', family='Arial')
plt.rc('font', size=10)
plt.rc('figure', figsize=(5,5))
S_HVT_Penetration = []
for iPenetration in range(len(penetration_list)):
    S_HVT_Penetration_temp = np.array(Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]])[:,0]
    S_HVT_Penetration.append(S_HVT_Penetration_temp)
plt.boxplot(S_HVT_Penetration)
plt.ylim([0.00, 100])
plt.legend(['Utilisation of HV transformer'])
plt.ylabel('Utilisation pecentage')
plt.xlabel('Penetration')
plt.xticks([1, 2, 3, 4, 5, 6], ['0', '20', '40', '60', '80', '100',])
plt.show()
plt.close()

## Fig.2: Distribution transformer maximum utilization level 

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.rc('font', family='Arial')
plt.rc('font', size=10)
plt.rc('figure', figsize=(5,5))
S_LVT_Penetration = []
for iPenetration in range(len(penetration_list)):
    S_LVT_Penetration_temp = np.array(Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]])[:,1]
    S_LVT_Penetration.append(S_LVT_Penetration_temp)
plt.boxplot(S_LVT_Penetration)
plt.ylim([0.00, 250])
plt.legend(['Utilisation of Dist. transformer'])
plt.ylabel('Utilisation pecentage')
plt.xlabel('Penetration')
plt.xticks([1, 2, 3, 4, 5, 6], ['0', '20', '40', '60', '80', '100',])
plt.show()
plt.close()

## Fig.3: Utilisation level for 22kV cables

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.rc('font', family='Arial')
plt.rc('font', size=10)
plt.rc('figure', figsize=(5,5))
Line_Util_Penetration = []
for iPenetration in range(len(penetration_list)):
    Line_Util_Penetration_temp = np.array(Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]])[:,2]
    Line_Util_Penetration.append(Line_Util_Penetration_temp)
plt.boxplot(Line_Util_Penetration)
plt.ylim([0.00, 120])
plt.legend(['Line utilisation'])
plt.ylabel('Utilisation pecentage')
plt.xlabel('Penetration')
plt.xticks([1, 2, 3, 4, 5, 6], ['0', '20', '40', '60', '80', '100',])
plt.show()
plt.close()

## Fig.4: Percentage of customers with voltage issues

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.rc('font', family='Arial')
plt.rc('font', size=10)
plt.rc('figure', figsize=(5,5))
Load_nonCompliance = []
for iPenetration in range(len(penetration_list)):
    Load_nonCompliance_temp = np.array(Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]])[:,3]
    Load_nonCompliance.append(Load_nonCompliance_temp )
plt.boxplot(Load_nonCompliance)
plt.legend(['Non_compliance'])
plt.ylabel('Non_compliance pecentage')
plt.xlabel('Penetration')
plt.xticks([1, 2, 3, 4, 5, 6], ['0', '20', '40', '60', '80', '100',])
plt.show()
plt.close()

## Fig.5: PV Curtailed Power in kW

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.rc('font', family='Arial')
plt.rc('font', size=10)
plt.rc('figure', figsize=(5,5))
PV_cutail = []
for iPenetration in range(len(penetration_list)):
    PV_cutail_temp = np.array(Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]])[:,4]
    PV_cutail.append(PV_cutail_temp )
plt.boxplot(PV_cutail)
plt.legend(['PV Curtailed Power'])
plt.ylabel('Curtailed (kW)')
plt.xlabel('Penetration')
plt.xticks([1, 2, 3, 4, 5, 6], ['0', '20', '40', '60', '80', '100',])
plt.show()
plt.close()

## Fig.6: PV Curtailed Power in Percentage

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.rc('font', family='Arial')
plt.rc('font', size=10)
plt.rc('figure', figsize=(5,5))
PV_percent = []
for iPenetration in range(len(penetration_list)):
    PV_percent_temp = 100*np.array(Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]])[:,4] / (1+np.array(Penetration_results_dct['penetration=%s'%penetration_list[iPenetration]])[:,5])
    PV_percent.append(PV_percent_temp )
plt.boxplot(PV_percent)
plt.legend(['PV Curtailed Percent'])
plt.ylabel('Curtailed (%)')
plt.xlabel('Penetration')
plt.xticks([1, 2, 3, 4, 5, 6], ['0', '20', '40', '60', '80', '100',])
plt.show()
plt.close()  